In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp
import scipy.stats as stats
from scipy.stats import ttest_rel

In [2]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

In [3]:
data2 = data.query('exclShopSales >=100 & shopSalesFromFuel >=100')
data2['log_exclShopSales'] =   np.log(data2['exclShopSales'])
data2['log_shopSalesFromFuel'] =   np.log(data2['shopSalesFromFuel'])
data2['logP_by_P0_ADO'] = np.log(data2['ADO_ Approved Price']/data2['ADO_ Marker Price'])
data2['logP_by_P0_UL95'] = np.log(data2['UL 95_ Approved Price']/data2['UL 95_ Marker Price'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [4]:
#elasticity calculation :: forcing the intercept to zero 
def regRes_gen_const(site,x,y,prefix):
    cols = ['const','slope','rvalu','pvalue','count']
    cols_ref = list(map(lambda x: x+prefix,cols))
    try:
        # filter all data of the site between 5th & 95th precentile 
        #q_low,q_up = site[y].quantile([0.02,0.98]).values.round().tolist()
        #xf = site.loc[(site[y] >= q_low)& (site[y] <= q_up)]

        #q_low,q_up = site[x].quantile([0.02,0.98]).values.round().tolist()
        #xf = site.loc[(site[x] >= q_low)& (site[x] <= q_up)]

        x_train,y_train = site[x], site[y]
        x_train = sm.add_constant(x_train)

        model = sm.OLS(y_train,x_train).fit()
        const,slope,rvalu,pvalue = model.params[0],model.params[1],round(model.rsquared,2),model.pvalues[1]
        
        #slope,intercept,rvalu,pvalue,stderr= stats.linregress(xf[x],xf[y])

        return(pd.Series([const,slope,rvalu,pvalue,len(x_train)],index=cols_ref))
    except Exception as e:
        print(e)
        return(pd.Series([0,0,0,0,len(x_train)],index=cols_ref))

In [8]:
# Excl Shop sales volume wrt ADO price position 
x_var = 'logP_by_P0_ADO'
ref = '_ADO'
data3 = data2.query('logP_by_P0_ADO != 0')[['Site Name','PBL','log_exclShopSales','logP_by_P0_ADO']].reset_index()
regExclShopSalesEl_ADO = data3.groupby(['Site Name','PBL']).apply(lambda site: \
                                                regRes_gen_const(site,x_var,'log_exclShopSales',ref)).reset_index()

# Shop sales From Fuel Sales volume wrt ADO price position 
x_var = 'logP_by_P0_ADO'
ref = '_ADO'
data3 = data2.query('logP_by_P0_ADO != 0')[['Site Name','PBL','log_shopSalesFromFuel','logP_by_P0_ADO']].reset_index()
regShopSalesFromFuelEl_ADO = data3.groupby(['Site Name','PBL']).apply(lambda site: \
                                                regRes_gen_const(site,x_var,'log_shopSalesFromFuel',ref)).reset_index()

# Excl Shop sales volume wrt UL price position 
x_var = 'logP_by_P0_UL95'
ref = '_UL'
data3 = data2.query('logP_by_P0_UL95 != 0')[['Site Name','PBL','log_exclShopSales','logP_by_P0_UL95']].reset_index()
regExclShopSalesEl_UL = data3.groupby(['Site Name','PBL']).apply(lambda site: \
                                                regRes_gen_const(site,x_var,'log_exclShopSales',ref)).reset_index()

# Shop sales From Fuel Sales volume wrt ADO price position 
x_var = 'logP_by_P0_UL95'
ref = '_UL'
data3 = data2.query('logP_by_P0_ADO != 0')[['Site Name','PBL','log_shopSalesFromFuel','logP_by_P0_UL95']].reset_index()
regShopSalesFromFuelEl_UL = data3.groupby(['Site Name','PBL']).apply(lambda site: \
                                                regRes_gen_const(site,x_var,'log_shopSalesFromFuel',ref)).reset_index()

regExclShopSalesEl_ADO['EL_label'] = 'ExclShopSalesEL_ADO'
regShopSalesFromFuelEl_ADO['EL_label'] = 'ShopSalesFromFuelEl_ADO'
regExclShopSalesEl_UL['EL_label'] = 'ExclShopSalesEl_UL'
regShopSalesFromFuelEl_UL['EL_label'] = 'ShopSalesFromFuelEl_UL'

In [9]:
cols = ['Site Name','PBL', 'const', 'slope', 'rvalu','pvalue','count', 'EL_label']
regExclShopSalesEl_ADO.columns = cols
regShopSalesFromFuelEl_ADO.columns = cols
regExclShopSalesEl_UL.columns = cols
regShopSalesFromFuelEl_UL.columns = cols

In [10]:
q_labels = ['High','Medium','Low','Very Low']
regExclShopSalesEl_ADO['EL_cat'] = pd.qcut(regExclShopSalesEl_ADO["slope"],q=4,labels=q_labels)
regShopSalesFromFuelEl_ADO['EL_cat'] = pd.qcut(regShopSalesFromFuelEl_ADO["slope"],q=4,labels=q_labels)
regExclShopSalesEl_UL['EL_cat'] = pd.qcut(regExclShopSalesEl_UL["slope"],q=4,labels=q_labels)
regShopSalesFromFuelEl_UL['EL_cat'] = pd.qcut(regShopSalesFromFuelEl_UL["slope"],q=4,labels=q_labels)

In [25]:
regExclShopSalesEl_ADO.groupby(['EL_cat'])['slope'].describe().unstack().reset_index()

EL_cat  count  mean  std    min   25%   50%   75%   max
0      High  49.00 -2.14 1.63 -11.29 -2.32 -1.58 -1.30 -0.94
1    Medium  49.00 -0.56 0.30  -0.94 -0.84 -0.57 -0.28 -0.06
2       Low  49.00  0.29 0.21  -0.04  0.13  0.26  0.45  0.72
3  Very Low  49.00  1.74 1.00   0.77  1.03  1.40  2.17  5.34

In [26]:
regShopSalesFromFuelEl_ADO.groupby(['EL_cat'])['slope'].describe().unstack().reset_index()

EL_cat  count  mean  std   min   25%   50%   75%   max
0      High  49.00 -3.59 1.81 -8.24 -4.66 -3.49 -1.98 -1.50
1    Medium  49.00 -0.77 0.33 -1.49 -0.96 -0.80 -0.54 -0.19
2       Low  49.00  0.62 0.49 -0.18  0.29  0.57  1.05  1.52
3  Very Low  49.00  3.90 5.34  1.52  2.13  2.77  3.50 38.41

In [11]:
df = pd.concat([regExclShopSalesEl_ADO,regExclShopSalesEl_UL,regShopSalesFromFuelEl_ADO,\
               regShopSalesFromFuelEl_UL],axis=0,ignore_index=True)

In [12]:
df.to_excel('Elasticity_From_shopSales_ADO_UL_2.xlsx',index=False)

In [13]:
df.to_pickle(r'Elasticity_From_shopSales_ADO_UL_2.pkl')

In [16]:
cols_sel = pd.read_excel(r'cols_select.xlsx')

cols = cols_sel.query('is_sel ==1')['colName'].tolist()

In [20]:
data2.loc[:,cols].to_excel(r'tab_viz_elasticty.xlsx')